In [ ]:
!pip install numpy==1.26.4
#o gensim não é compatível (pelo menos até o momento) com a versão 2.x do numpy
#talvez seja necessário reiniciar a sessão.

In [ ]:
#verificar versão do numpy. Se estiver acima de 2.x terá problemas com gensim.
import numpy
print(numpy.__version__)

1.26.4


In [ ]:
!pip install --upgrade gensim

In [ ]:
from gensim.models import KeyedVectors

# Carregando dados


Para esta atividade nós utilizaremos vetores de palavras, também conhecidos como *embeddings*, para lingua portuguesa fornecidos pelo [NILC](http://www.nilc.icmc.usp.br/nilc/index.php). Nós utilizaremos o embedding de 50 dimensões treinado com o algoritmo Word2Vec (Continous Bag of Words) que pode ser encontrado [aqui](http://www.nilc.icmc.usp.br/embeddings) sob a licensa [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/). Para evitar problemas de mémoria utilizaremos apenas as 200 mil palavras mais comum.

In [ ]:
!curl  https://raw.githubusercontent.com/alan-barzilay/NLPortugues/master/Semana%2004/data/word2vec_200k.txt --output 'word2vec_200k.txt'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 92.0M  100 92.0M    0     0  12.6M      0  0:00:07  0:00:07 --:--:-- 22.7M


In [ ]:
# Carrega word2vec
model = KeyedVectors.load_word2vec_format("word2vec_200k.txt")

# Similaridade e Distância Cosseno

Como comentamos em sala de aula, podemos considerar as palavras como pontos num espaço n-dimensional e podemos examinar a proximidade delas através da similaridade cosseno:
$$s = \frac{u \cdot v}{||u|| ||v||}, \textrm{ onde } s \in [-1, 1] $$


## <font color='blue'>Questão 1 </font>
Palavras [polissemicas](https://pt.wikipedia.org/wiki/Polissemia) e [homônimas](https://pt.wikipedia.org/wiki/Hom%C3%B3nimo) são palavras que possuem mais de um significado.


Utilizando a função `model.most_similar(positive = palavra1)`, que retorna uma lista das palavras mais similares à palavra1, encontre uma palavra que possua múltiplos significados. Observe que na sua lista de 10 palavras mais similares existam palavras relacionadas a mais de um dos seus significados, lembre-se de consultar sua [documentação](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.most_similar).

Por exemplo, a palavra "manga" possui na sua lista de 10 palavras mais similares as palavras "gola" e "lapela" (que estão relacionadas ao significado de manga de uma camiseta) e a palavra "maçã" (que está relacionada ao significado da fruta manga).



In [ ]:
palavra = "banco"
similar_words = model.most_similar(positive=[palavra], topn=10)

for word, score in similar_words:
    print(word, score)

observatório 0.7820174098014832
governo 0.781789243221283
consórcio 0.7666717171669006
comitж 0.7602732181549072
orуamento 0.7499715089797974
tesouro 0.7444049119949341
mercado 0.7383556962013245
setor 0.7276566028594971
monopólio 0.7259189486503601
cine-theatro 0.7205812931060791


# Sinônimos e Antônimos


As vezes é mais intuitivo trabalhar com uma medida de distancia ao invés da similaridade cosseno, para isso vamos utilizar a distancia cosseno que é simplesmente 1 - Similaridade Cosseno.

## <font color='blue'>Questão 2 </font>


Usando a função [`model.distance(palavra1,palavra2)`](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.distance), encontre 3 palavras onde as palavras p1 e p2 são sinônimas e p1 e p3 são antônimas mas `distance(p1,p3)` < `distance(p1,p2)`.

Proponha uma explicação do porque esse resultado contraintuitivo acontece.






In [ ]:
p1 = "bom"
p2 = "ótimo"   # sinônimo
p3 = "ruim"    # antônimo

dist_syn = model.distance(p1, p2)
dist_ant = model.distance(p1, p3)

print("Distância bom-ótimo:", dist_syn)
print("Distância bom-ruim:", dist_ant)

Distância bom-ótimo: 0.17712664604187012
Distância bom-ruim: 0.23797184228897095


Word2Vec captura contexto, não significado direto.

“Bom” e “ruim” aparecem em contextos muito semelhantes, como:

“O filme é bom/ruim”

“A comida é boa/ruim”

Já “ótimo” pode aparecer em contextos mais específicos:

“Ótimo desempenho”, “Ótimo filme”

Portanto, vetores de antônimos podem acabar mais próximos do que alguns sinônimos raros ou mais específicos.

# Analogias

Existem algumas analogias famosas realizadas por vetores de palavras. O exemplo mais famoso é provavelmente "man : king :: woman : x", onde x é *queen*.

Para formular analogias vamos utilizar a função `most_similar()` que busca as palavras mais similares as listas em  `positive` e mais dissimilares as listadas em  `negative`. Para mais detalhes recomendamos consultar a sua [documentação](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.most_similar).




In [ ]:
model.most_similar(positive=['mulher', 'engenheiro'], negative=['homem'])

[('engenheira', 0.7883446216583252),
 ('investigadora', 0.7415961623191833),
 ('ex-funcionária', 0.7373332977294922),
 ('enfermeira', 0.7346670627593994),
 ('funcionária', 0.7172971367835999),
 ('bibliotecária', 0.7110162377357483),
 ('arquiteta', 0.7099220156669617),
 ('empresária', 0.7055575847625732),
 ('ex-diretora', 0.7055395841598511),
 ('professora', 0.697813868522644)]

## <font color='blue'>Questão 3 </font>
Encontre analogias que funcionam, ou seja, que a palavra esperada está no topo da lista.

Descreva sua analogia na seguinte forma:
x:y :: a:b



In [ ]:
# rei : rainha :: homem : mulher

model.most_similar(positive=['rainha','homem'], negative=['rei'])
# Esperado: mulher

[('crianção', 0.6878494620323181),
 ('fazendeira', 0.6852145195007324),
 ('virgem', 0.679385781288147),
 ('adúltera', 0.6674581170082092),
 ('peixinha', 0.6659393310546875),
 ('moça', 0.6614577174186707),
 ('menage', 0.6604392528533936),
 ('ganancia', 0.6598883867263794),
 ('loira', 0.6595181822776794),
 ('barriga', 0.652824342250824)]

In [ ]:
# paraná : brasil :: califórnia : estados_unidos

model.most_similar(positive=['california','brasil'], negative=['parana'])
# Esperado: estados_unidos

[('filmore', 0.6279376745223999),
 ('moscow', 0.6223193407058716),
 ('lyceum', 0.6135118007659912),
 ('psych', 0.6131197214126587),
 ('brixton', 0.6120437979698181),
 ('deadwood', 0.6099486351013184),
 ('eggsy', 0.6092063188552856),
 ('morning', 0.6085426807403564),
 ('battler', 0.5998415946960449),
 ('geils', 0.5996437072753906)]

In [ ]:
# cachorro : filhote :: gato : gatinho

model.most_similar(positive=['filhote','gato'], negative=['cachorro'])
# Esperado: gatinho

[('frêmito', 0.8587124347686768),
 ('fiapo', 0.8570783138275146),
 ('tonel', 0.8188515305519104),
 ('buquê', 0.8162720799446106),
 ('cálice', 0.8151410818099976),
 ('pássaro', 0.8137834072113037),
 ('novelo', 0.8087284564971924),
 ('pudim', 0.801990807056427),
 ('chapéu', 0.8018507361412048),
 ('veado', 0.8003045916557312)]

## <font color='blue'>Questão 4 </font>
Encontre analogias que **Não** funcionam.

Descreva o resultado esperado da sua analogia na seguinte forma:
x:y :: a:b

E indique o valor errado de b encontrado



# Viés e preconceito adquirido

Como estes vetores são aprendidos a partir de documentos produzidos pela nossa sociedade, ele pode vir a capturar alguns preconceitos e desigualdades presentes na nossa sociedade. É importante estar ciente desse viés de nossos vetores e dos seus perigos, aplicações que utilizam esses modelos podem acabar perpetuando e até mesmo exacerbando desigualdades sociais.

Por exemplo, uma analogia problemática capturada:



In [ ]:
model.most_similar(positive=['negro', 'rico'], negative=['pobre'])

[('branco', 0.663209080696106),
 ('alegre/rs', 0.6620162725448608),
 ('braga-fc', 0.6464027762413025),
 ('sporting-fc', 0.6254758238792419),
 ('côvo', 0.6254613995552063),
 ('alegre-rs', 0.6199708580970764),
 ('vermelho', 0.612277090549469),
 ('covo', 0.604120671749115),
 ('cirílicos', 0.6022458672523499),
 ('benfica-fc', 0.5965930819511414)]

In [ ]:
# Exemplo: associação de gênero com cargo
resultado = model.most_similar(positive=['mulher', 'líder'], negative=['homem'])
print("Homem : líder :: mulher : ?", resultado)
# Esperado problemático: palavras como 'secretária' ou termos femininos estereotipados

Homem : líder :: mulher : ? [('sucessora', 0.7438196539878845), ('dissidência', 0.7324091196060181), ('interlocutora', 0.7215778231620789), ('antecessora', 0.7209436893463135), ('conterrânea', 0.7189176678657532), ('supremacia', 0.712788999080658), ('ex-governadora', 0.707196056842804), ('indicaусo', 0.7064744830131531), ('desfiliação', 0.7056934237480164), ('apresentaусo', 0.7051743865013123)]


In [ ]:
# Exemplo: associação de profissão com gênero
resultado = model.most_similar(positive=['enfermeira', 'homem'], negative=['mulher'])
print("Mulher : enfermeira :: homem : ?", resultado)
# Esperado problemático: palavras como 'engenheiro', refletindo estereótipos de gênero

Mulher : enfermeira :: homem : ? [('detento', 0.8834566473960876), ('rapaz', 0.8791738152503967), ('pescador', 0.8454643487930298), ('taxista', 0.8408356308937073), ('garoto', 0.8356291651725769), ('indivíduo', 0.8337016701698303), ('ex-condenado', 0.8319786787033081), ('tudo-nada', 0.828228771686554), ('mendigo', 0.8273341655731201), ('rapazinho', 0.8230759501457214)]


In [ ]:
# Exemplo: associação racial com profissões ou status
resultado = model.most_similar(positive=['negro', 'professor'], negative=['branco'])
print("Branco : professor :: negro : ?", resultado)
# Esperado problemático: pode gerar profissões ou termos estereotipados

Branco : professor :: negro : ? [('praticante', 0.7777761816978455), ('ex-professor', 0.7484919428825378), ('imunologista', 0.7463474869728088), ('inventor', 0.7448247671127319), ('mentor', 0.7401202321052551), ('filósofo', 0.7380432486534119), ('orientador', 0.7369438409805298), ('estudioso', 0.7318746447563171), ('teólogo', 0.7314227819442749), ('tutor', 0.7310663461685181)]


In [ ]:
# Exemplo: associação de adjetivos positivos/negativos com gênero
resultado = model.most_similar(positive=['mulher', 'inteligente'], negative=['homem'])
print("Homem : inteligente :: mulher : ?", resultado)
# Esperado problemático: pode gerar adjetivos relacionados a aparência ou emoções, reforçando estereótipos

Homem : inteligente :: mulher : ? [('novata', 0.7025105953216553), ('interesseira', 0.6901221871376038), ('sadia', 0.6841514706611633), ('talentosa', 0.6800923347473145), ('aventureira', 0.6706680059432983), ('estudiosa', 0.6655993461608887), ('super-heroína', 0.6474816799163818), ('gordinha', 0.6469574570655823), ('dinã¢mica', 0.6433157324790955), ('ex-atriz', 0.641520082950592)]


## <font color='blue'>Questão 5 </font>

Utiliza a função `most_similar()` para encontrar um outro caso de viés adquirido pelos vetores e explique brevemente o tipo de viés encontrado.



In [ ]:
# Investigando viés de idade associado a profissão ou adjetivos
resultado = model.most_similar(positive=['idoso', 'tecnologia'], negative=['jovem'])
print("Jovem : tecnologia :: idoso : ?", resultado)
# Possível resultado problemático: palavras como 'difícil', 'lentidão', 'tradicional'
# Tipo de viés: estereótipo de que idosos têm dificuldade com tecnologia

Jovem : tecnologia :: idoso : ? [('monitoramento', 0.7619328498840332), ('toxicologia', 0.7600721120834351), ('sáude', 0.7595340013504028), ('saúde', 0.7494494318962097), ('saneamento', 0.7397071719169617), ('salubridade', 0.7354581356048584), ('geotecnia', 0.7241795063018799), ('odontologia', 0.7211127877235413), ('informática', 0.7187449336051941), ('microbiologia', 0.7182848453521729)]


In [ ]:
# Investigando viés religioso associado a moral ou comportamento
resultado = model.most_similar(positive=['cristão', 'honesto'], negative=['muçulmano'])
print("Muçulmano : honesto :: Cristão : ?", resultado)
# Possível resultado problemático: palavras que reforçam preconceitos sobre integridade
# Tipo de viés: religioso

Muçulmano : honesto :: Cristão : ? [('sincero', 0.860211193561554), ('generoso', 0.8267276287078857), ('amável', 0.8082507252693176), ('afetuoso', 0.8026919960975647), ('compreensivo', 0.7904291749000549), ('pretensioso', 0.7845579385757446), ('respeitoso', 0.78075110912323), ('simpático', 0.7800769805908203), ('mesquinho', 0.7781084775924683), ('cortês', 0.7760818600654602)]


In [ ]:
# Investigando viés de nacionalidade em profissões ou características
resultado = model.most_similar(positive=['americano', 'rico'], negative=['brasileiro'])
print("Brasileiro : rico :: Americano : ?", resultado)
# Possível resultado problemático: palavras que reforçam estereótipos de riqueza por nacionalidade
# Tipo de viés: nacionalidade/econômico

Brasileiro : rico :: Americano : ? [('marghera', 0.6235029697418213), ('schwäbisch', 0.5838977098464966), ('altenburger', 0.5732682347297668), ('alberta', 0.5563464760780334), ('huntsville', 0.5493906140327454), ('robecco', 0.5467824339866638), ('arnsberg', 0.5407811999320984), ('nürnberger', 0.535463273525238), ('palmyra', 0.518111526966095), ('alegre-rs', 0.5177686810493469)]


In [ ]:
# Investigando viés de gênero associado a adjetivos de personalidade
resultado = model.most_similar(positive=['mulher', 'ambicioso'], negative=['homem'])
print("Homem : ambicioso :: Mulher : ?", resultado)
# Possível resultado problemático: palavras como 'sensível', 'emocional'
# Tipo de viés: gênero/estereótipo de personalidade

Homem : ambicioso :: Mulher : ? [('novata', 0.6051635146141052), ('resseguradora', 0.5853472948074341), ('remistura', 0.5785679817199707), ('ex-atriz', 0.5731431245803833), ('obra-prima', 0.565626859664917), ('oneroso', 0.5655460357666016), ('atech', 0.5617077350616455), ('ergonómico', 0.5615898370742798), ('dinã¢mica', 0.5580483675003052), ('oprimeiro', 0.5560930371284485)]


## <font color='blue'>Questão 6 </font>

Qual é a possivel origem desses vieses? Tente explicar como eles podem ter sido capturados pelos vetores de palavras.